# Introdução


O mundo do futebol, um esporte globalmente aclamado, oferece vastas oportunidades para análises e insights valiosos por meio de técnicas avançadas de aprendizado de máquina. Neste projeto, explorou-se a aplicação dessas técnicas para analisar e classificar os resultados de partidas de futebol, com base em dados detalhados das temporadas de 2014 a 2017 obtidos através da plataforma "Kaggle".

Os conjuntos de dados abrangentes incluem informações sobre jogadores, clubes e resultados de partidas, levando em consideração variáveis como gols marcados, aproveitamento como mandante e visitante, número de gols, entre outros fatores determinantes (features), calculados posteriormente. A ênfase recai na criação de dois classificadores distintos: um utilizando Regressão Logística e outro baseado em Árvores de Decisão. Esses modelos visam prever os resultados (classes) de partidas com base em padrões históricos.

Ao longo da implementação desses classificadores, avaliou-se a eficácia de cada abordagem e comparou-se os resultados obtidos. A análise detalhada das matrizes de confusão proporcionará uma compreensão aprofundada do desempenho de cada modelo, contribuindo para insights significativos no contexto do futebol.

# Fonte

[Kaggle - Cartola FC Dataset](https://www.kaggle.com/datasets/schiller/cartolafc)

[Scikit Learn - LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)

[Scikit Learn - DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

# Código

## Tratamento dos dados

Inicialmente obtém-se os dados para anáise. Assim relaiza-se o upload das bases previamente baixadas, através do link disponibilizado. Utilizou-se os seguintes arquivos da pasta:

2017_scouts.csv, 2016_scouts.csv, 2017_atletas.csv, 2017_clubes.csv, 2017_partidas.csv, 2016_atletas.csv, 2016_clubes.csv, 2016_partidas.csv 2014_scouts.csv, 2015_scouts.csv, 2015_atletas.csv, 2015_clubes.csv, 2015_partidas.csv, 2014_atletas.csv, 2014_clubes.csv, 2014_partidas.csv

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import numpy as np
import pandas as pd
import io
import seaborn as sns
import matplotlib.pyplot as plt

# Obtenção dos jogadores de 2014/2015/2016/2017
scouts_2014   = pd.read_csv((io.StringIO(uploaded['2014_scouts.csv'].decode('utf-8'))))
scouts_2015   = pd.read_csv((io.StringIO(uploaded['2015_scouts.csv'].decode('utf-8'))))
scouts_2016   = pd.read_csv((io.StringIO(uploaded['2016_scouts.csv'].decode('utf-8'))))
scouts_2017   = pd.read_csv((io.StringIO(uploaded['2017_scouts.csv'].decode('utf-8'))))

# Obtenção dos dados das patidas de 2014/2015/2016/2017
partidas_2014 = pd.read_csv((io.StringIO(uploaded['2014_partidas.csv'].decode('utf-8'))))
partidas_2015 = pd.read_csv((io.StringIO(uploaded['2015_partidas.csv'].decode('utf-8'))))
partidas_2016 = pd.read_csv((io.StringIO(uploaded['2016_partidas.csv'].decode('utf-8'))))
partidas_2017 = pd.read_csv((io.StringIO(uploaded['2017_partidas.csv'].decode('utf-8'))))

# Obtenção dos dados dos atletas de 2014/2015/2016/2017
atletas_2014  = pd.read_csv((io.StringIO(uploaded['2014_atletas.csv'].decode('utf-8'))))
atletas_2015  = pd.read_csv((io.StringIO(uploaded['2015_atletas.csv'].decode('utf-8'))))
atletas_2016  = pd.read_csv((io.StringIO(uploaded['2016_atletas.csv'].decode('utf-8'))))
atletas_2017  = pd.read_csv((io.StringIO(uploaded['2017_atletas.csv'].decode('utf-8'))))

# Obtenção dos dados dos clubes de 2014/2015/2016/2017
clubes_2014   = pd.read_csv((io.StringIO(uploaded['2014_clubes.csv'].decode('utf-8'))))
clubes_2015   = pd.read_csv((io.StringIO(uploaded['2015_clubes.csv'].decode('utf-8'))))
clubes_2016   = pd.read_csv((io.StringIO(uploaded['2016_clubes.csv'].decode('utf-8'))))
clubes_2017   = pd.read_csv((io.StringIO(uploaded['2017_clubes.csv'].decode('utf-8'))))


Assim, pode-se observar o conteúdo de cada arquivo (exemplo):

In [ ]:
# Exemplo para a base de "scout" (jogadores/ desempenho)
scouts_2016

In [ ]:
# Exemplo para a base de partidas
partidas_2014

In [ ]:
# Exemplo para a base de atletas (clube/ posição)
atletas_2014

In [ ]:
# Exemplo para a base de clubes
clubes_2014

Nota-se que é necessário realizar um tratamento para obter um dataset completo e devidamente "nomeado" para cada partida de cada rodada dos campeonatos, descritos com os respectivos placares e times.

In [ ]:
# Agrupar informações das partidas e clubes ("labels")

#Agrupa todos os times que jogaram nas temporadas 2014/2015/2016/2017
import pandas as pd

clubes = pd.concat([clubes_2014, clubes_2015, clubes_2016, clubes_2017], ignore_index=True)
clubes = clubes.drop_duplicates(subset='nome')
clubes

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2014
df_2014 = partidas_2014.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2014 = df_2014.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2014 = df_2014[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2014['YEAR'] = '2014'
df_2014.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2015
df_2015 = partidas_2015.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2015 = df_2015.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2015 = df_2015[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2015['YEAR'] = '2015'
df_2015.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2016
df_2016 = partidas_2016.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2016 = df_2016.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2016 = df_2016[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2016['YEAR'] = '2016'
df_2016.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2017
df_2017 = partidas_2017.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2017 = df_2017.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2017 = df_2017[['rodada_id', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2017['YEAR'] = '2017'
df_2017.rename(columns = {'nome': 'MANDANTE', 'rodada_id': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE','nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Join 2014/2015/2016/2017
df0 = df_2016
df = pd.concat([df0, df_2017, df_2014, df_2015], ignore_index=True)
df = df[df['GOLS_MANDANTE'].notna()] # há alguns jogos sem informação (sem o placar)

Após tratar os dados e consolidar todas as informações em um único dataset, calculou-se a o aproveitamento, quantidade de gols feitos e tomados, dentro e fora de casa (mandante e visitante).

In [ ]:
# Listagem do aproveitamento dentro e fora de casa para cada time durante o campeonato
l_times_2014 = clubes_2014['nome'].tolist()
l_times_2015 = clubes_2015['nome'].tolist()
l_times_2016 = clubes_2016['nome'].tolist()
l_times_2017 = clubes_2017['nome'].tolist()
l_times = list(set(l_times_2014 + l_times_2015 + l_times_2016 + l_times_2017))
l = []

# Passa por cada jogo e analisa o resultado
for i in range(len(l_times)):
  j_m = 0
  j_v = 0
  v = 0
  u = 0
  g_m = 0
  g_v = 0
  g_m_t = 0
  g_v_t = 0
  # Analisa todos os jogos de um time como mandante e acumula a quantidade de vitórias, gols feitos e tomados
  df_time = df[(df.MANDANTE == l_times[i])]
  df_time.reset_index(inplace=True, drop=True)
  for j in range(len(df_time)):
    j_m += 1
    g_m += df_time.at[j, 'GOLS_MANDANTE']
    g_m_t += df_time.at[j, 'GOLS_VISITANTE']
    if df_time.at[j, 'GOLS_MANDANTE'] > df_time.at[j, 'GOLS_VISITANTE']:
      v += 1

  # Analisa todos os jogos de um time como visitante e acumula a quantidade de vitórias, gols feitos e tomados
  df_time = df[(df.VISITANTE == l_times[i])]
  df_time.reset_index(inplace=True, drop=True)
  for k in range(len(df_time)):
    j_v += 1
    g_v += df_time.at[k, 'GOLS_VISITANTE']
    g_v_t += df_time.at[k, 'GOLS_MANDANTE']
    if df_time.at[k, 'GOLS_MANDANTE'] < df_time.at[k, 'GOLS_VISITANTE']:
      u += 1

  # Cálculo do número de jogos por time
  n_j = j_m + j_v

  # Cálculo do número de vitórias
  n_v = u + v

  # Cálculo do aproveitamento como mandante (n° de vitórias/ quantidade de partidas)
  ap_man = v/n_j

  # Cálculo do aproveitamento como visitante (n° de vitórias/ quantidade de partidas)
  ap_vis = u/n_j

  # Cálculo do número de gols feitos
  g_f = g_m + g_v

  # Cálculo do número de gols tomados
  g_t = g_m_t + g_v_t

  # Cria uma lista para cada time seguido dos aproveitamentos (mandante e visitante,respectivamente)
  l.append([l_times[i], n_j, n_v, ap_man, ap_vis, g_f, g_t])

# DataFrame resultante
df_ap = pd.DataFrame(l, columns=['Time', 'N°_jogos', 'N°_vitórias', 'Aproveitamento_Mandante', 'Aproveitamento_Visitante', 'Gols_feitos', 'Gols_tomados'])
df_ap

Tendo as características (features) a serem consideradas (df_ap) e os resultados dos jogos (df), pode-se combinar essas informações a fim de classificar cada partida e treinar este classificador.

In [ ]:
# Juntando as informações das duas bases

df_f = pd.merge(df, df_ap, left_on='MANDANTE', right_on='Time')
df_f = pd.merge(df_f, df_ap, left_on='VISITANTE', right_on='Time', suffixes=('_MANDANTE', '_VISITANTE'))

# Definindo o resultado do jogo
df_f['Resultado'] = np.where(df_f['GOLS_MANDANTE'] > df_f['GOLS_VISITANTE'], 'Vitória_MANDANTE', np.where(df_f['GOLS_MANDANTE'] < df_f['GOLS_VISITANTE'], 'Vitória_VISITANTE', 'Empate'))

Antes de iniciar a classificação e treinamentos, pode-se observar o dataset final:

In [ ]:
df_f

## Classificador

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Definição das features
feat = ['Aproveitamento_Mandante_MANDANTE', 'Aproveitamento_Visitante_MANDANTE', 'Gols_feitos_MANDANTE', 'Gols_tomados_MANDANTE', 'Aproveitamento_Mandante_VISITANTE', 'Aproveitamento_Visitante_VISITANTE', 'Gols_feitos_VISITANTE', 'Gols_tomados_VISITANTE']

# Divisão dos dados em treino e teste através do train_test_split
X = df_f[feat]
y = df_f['Resultado']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

A Regressão Logística é uma técnica comumente empregada em aprendizado de máquina para resolver problemas de classificação binária. Ela é utilizada para prever se uma instância pertence a uma das duas classes possíveis.

No contexto do código fornecido, a Regressão Logística é aplicada para antecipar o desfecho de um jogo de futebol, determinando se o time que joga em casa terá um resultado positivo ou negativo,, ou seja, está sendo considerado apenas vitória ou derrota.

In [ ]:
# Treinando o modelo LogisticRegression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Desempenho do modelo LogisticRegression
y_pred = log_reg.predict(X_test)
ac = accuracy_score(y_test, y_pred)
r_class = classification_report(y_test, y_pred)

print(f'Acurácia: {ac:.2f}')
print('Relatório de Classificação:\n', r_class)

### Decision Tree

A Árvore de Decisão é uma ferramenta poderosa no campo de aprendizado de máquina, sendo utilizada no código fornecido para prever os resultados de partidas de futebol, especificamente, categorizando se um time mandante será vitorioso, derrotado ou se a partida resultará em empate (neste caso, está sendo considerado as três classes possíveis).

Este modelo é especialmente útil para identificar padrões intricados nos dados, oferecendo ao mesmo tempo compreensibilidade ao ilustrar as escolhas feitas pelo modelo. Cada ponto de decisão na árvore (nó) reflete uma escolha fundamentada em uma característica específica, e os ramos delineiam as diversas consequências dessas decisões.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Treinando o modelo DecisionTree
d_tree = DecisionTreeClassifier(random_state=42)
d_tree.fit(X_train, y_train)

# Desempenho do modelo DecisionTree
y_pred_d_tree = d_tree.predict(X_test)
ac_d_tree = accuracy_score(y_test, y_pred_d_tree)
r_d_tree = classification_report(y_test, y_pred_d_tree)

print(f'Acurácia: {ac_d_tree:.2f}')
print("Relatório de Classificação:\n", r_d_tree)

### Matriz de Confusão

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Matriz de confusão
def plot_confusion_matrix(ax, y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax, vmin=0, vmax=100)  # Ajuste as escalas de cor aqui
    ax.set_title(title)
    ax.set_xlabel('Previsão do Modelo')
    ax.set_ylabel('Verdadeiro Resultado')

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Matriz de confusão para LogisticRegression
plot_confusion_matrix(axs[0], y_test, y_pred, 'Matriz de Confusão - Regressão Logística')

# Matriz de confusão para DecisionTree
plot_confusion_matrix(axs[1], y_test, y_pred_decision_tree, 'Matriz de Confusão - Árvore de Decisão')

plt.tight_layout()
plt.show()

# Análise dos resultados e conclusão (Primeira Versão)


Analisando os resultados e matrizes obtidas pode-se concluir que ambos os classificadores obtiveram resultado razoável para o conjunto de dados e o contexto aplicado, visto que obtiveram uma acurácia de 55% e 60% (Regressão logística e Árvore de Decisão, respectivamente). Observando especificamente o primeiro modelo, pode-se notar que uma de suas limitações é o fato de analisar somente dois casos (classes) possíveis, a vitória do time mandante ou vitória do time visitante (derrota do time mandante), o que pode ofuscar e dificultar parte do treinamento e obtenção de um padrão. É possível notar, também, que para os casos de vitória do mandante, o classificador obeteve um desempenho consideraelmente excepcional, obtendo um recall de 96%, o que se distingue para o caso oposto com um recall de 24%. Agora, para o caso com melhor resultado, se desenvolveu o classificador através das três classes possíveis, acrescentando, então, a situação de "Empate". Além disso, seguindo o mesmo padrão do outro analisado, a árvore de decisões apresentou uma maior precisão para o caso mandante (68% de precisão e 73% de recall) do que para o caso visitante (52% de precisão e 42% de recall), revelando-se, então, mais difícil de se prever com as "features" disponíveis.

Para finalizar, dessa forma, nota-se algumas características do contexto observado e melhorias possíveis para o código como: desenvolviemnto de outros modelos de classificação e adição de caracterísiticas (desempenho dos jogadores, histórico de confrontos, etc), assim como, a possibilidade de times apresentarem um menor volume de partidas, visto que há a questão do acesso e rebaixamento entre as séries do campeonato brasileiro. Além de reforçar que uma partida de futebol possui fatores externos à resultados diretos e estatísticas que influnciam diretamente o resultado de uma partida.

# Código (2.0)


## Tratamento dos dados

Inicialmente obtém-se os dados para anáise. Assim relaiza-se o upload das bases previamente baixadas, através do link disponibilizado. Utilizou-se os seguintes arquivos da pasta:

2017_scouts.csv, 2016_scouts.csv, 2017_atletas.csv, 2017_clubes.csv, 2017_partidas.csv, 2016_atletas.csv, 2016_clubes.csv, 2016_partidas.csv 2014_scouts.csv, 2015_scouts.csv, 2015_atletas.csv, 2015_clubes.csv, 2015_partidas.csv, 2014_atletas.csv, 2014_clubes.csv, 2014_partidas.csv

In [ ]:
pip install pandas

In [ ]:
pip install seaborn

In [ ]:
pip install matplotlib

In [ ]:
# Caminho para a pasta onde os arquivos estão
path = 'C:/Users/gabriel.fuziama/Desktop/TG/archive/'

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Obtenção dos jogadores de 2014/2015/2016/2017
scouts_2014 = pd.read_csv(path + '2014_scouts.csv')
scouts_2015 = pd.read_csv(path + '2015_scouts.csv')
scouts_2016 = pd.read_csv(path + '2016_scouts.csv')
scouts_2017 = pd.read_csv(path + '2017_scouts.csv')

# Obtenção dos dados das partidas de 2014/2015/2016/2017
partidas_2014 = pd.read_csv(path + '2014_partidas.csv')
partidas_2015 = pd.read_csv(path + '2015_partidas.csv')
partidas_2016 = pd.read_csv(path + '2016_partidas.csv')
partidas_2017 = pd.read_csv(path + '2017_partidas.csv')

# Obtenção dos dados dos atletas de 2014/2015/2016/2017
atletas_2014 = pd.read_csv(path + '2014_atletas.csv')
atletas_2015 = pd.read_csv(path + '2015_atletas.csv')
atletas_2016 = pd.read_csv(path + '2016_atletas.csv')
atletas_2017 = pd.read_csv(path + '2017_atletas.csv')

# Obtenção dos dados dos clubes de 2014/2015/2016/2017
clubes_2014 = pd.read_csv(path + '2014_clubes.csv')
clubes_2015 = pd.read_csv(path + '2015_clubes.csv')
clubes_2016 = pd.read_csv(path + '2016_clubes.csv')
clubes_2017 = pd.read_csv(path + '2017_clubes.csv')


Assim, pode-se observar o conteúdo de cada arquivo (exemplo):

Nota-se que é necessário realizar um tratamento para obter um dataset completo e devidamente "nomeado" para cada partida de cada rodada dos campeonatos, descritos com os respectivos placares e times.

In [ ]:
# Agrupar informações das partidas e clubes ("labels")

#Agrupa todos os times que jogaram nas temporadas 2014/2015/2016/2017
import pandas as pd

clubes = pd.concat([clubes_2014, clubes_2015, clubes_2016, clubes_2017], ignore_index=True)
clubes = clubes.drop_duplicates(subset='nome')
clubes

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2014
df_2014 = partidas_2014.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2014 = df_2014.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2014 = df_2014[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2014['YEAR'] = '2014'
df_2014.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2015
df_2015 = partidas_2015.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2015 = df_2015.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2015 = df_2015[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2015['YEAR'] = '2015'
df_2015.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2016
df_2016 = partidas_2016.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2016 = df_2016.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2016 = df_2016[['rodada', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2016['YEAR'] = '2016'
df_2016.rename(columns = {'nome': 'MANDANTE', 'rodada': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE', 'nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Substitui os ids pelos nomes e resultados específicos para cada jogo de cada rodada no ano de 2017
df_2017 = partidas_2017.set_index('clube_casa_id').join(clubes.set_index('id'), rsuffix='_casa')
df_2017 = df_2017.set_index('clube_visitante_id').join(clubes.set_index('id'), rsuffix='_visitante')
df_2017 = df_2017[['rodada_id', 'nome', 'placar_oficial_mandante', 'nome_visitante', 'placar_oficial_visitante']]
df_2017['YEAR'] = '2017'
df_2017.rename(columns = {'nome': 'MANDANTE', 'rodada_id': 'RODADA', 'placar_oficial_mandante': 'GOLS_MANDANTE','nome_visitante': 'VISITANTE', 'placar_oficial_visitante': 'GOLS_VISITANTE'}, inplace = True)

# Join 2014/2015/2016/2017
df0 = df_2016
df = pd.concat([df0, df_2017, df_2014, df_2015], ignore_index=True)
df = df[df['GOLS_MANDANTE'].notna()] # há alguns jogos sem informação (sem o placar)
df

Após tratar os dados e consolidar todas as informações em um único dataset, calculou-se a o aproveitamento, quantidade de gols feitos e tomados, dentro e fora de casa (mandante e visitante).

In [ ]:
# Listagem do aproveitamento dentro e fora de casa para cada time durante o campeonato
l_times_2014 = clubes_2014['nome'].tolist()
l_times_2015 = clubes_2015['nome'].tolist()
l_times_2016 = clubes_2016['nome'].tolist()
l_times_2017 = clubes_2017['nome'].tolist()
l_times = list(set(l_times_2014 + l_times_2015 + l_times_2016 + l_times_2017))
l = []

# Passa por cada jogo e analisa o resultado
for i in range(len(l_times)):
  j_m = 0
  j_v = 0
  v = 0
  u = 0
  g_m = 0
  g_v = 0
  g_m_t = 0
  g_v_t = 0
  # Analisa todos os jogos de um time como mandante e acumula a quantidade de vitórias, gols feitos e tomados
  df_time = df[(df.MANDANTE == l_times[i])]
  df_time.reset_index(inplace=True, drop=True)
  for j in range(len(df_time)):
    j_m += 1
    g_m += df_time.at[j, 'GOLS_MANDANTE']
    g_m_t += df_time.at[j, 'GOLS_VISITANTE']
    if df_time.at[j, 'GOLS_MANDANTE'] > df_time.at[j, 'GOLS_VISITANTE']:
      v += 1

  # Analisa todos os jogos de um time como visitante e acumula a quantidade de vitórias, gols feitos e tomados
  df_time = df[(df.VISITANTE == l_times[i])]
  df_time.reset_index(inplace=True, drop=True)
  for k in range(len(df_time)):
    j_v += 1
    g_v += df_time.at[k, 'GOLS_VISITANTE']
    g_v_t += df_time.at[k, 'GOLS_MANDANTE']
    if df_time.at[k, 'GOLS_MANDANTE'] < df_time.at[k, 'GOLS_VISITANTE']:
      u += 1

  # Cálculo do número de jogos por time
  n_j = j_m + j_v

  # Cálculo do número de vitórias
  n_v = u + v

  # Cálculo do aproveitamento como mandante (n° de vitórias/ quantidade de partidas)
  ap_man = v/n_j

  # Cálculo do aproveitamento como visitante (n° de vitórias/ quantidade de partidas)
  ap_vis = u/n_j

  # Cálculo do número de gols feitos
  g_f = g_m + g_v

  # Cálculo do número de gols tomados
  g_t = g_m_t + g_v_t

  # Cria uma lista para cada time seguido dos aproveitamentos (mandante e visitante,respectivamente)
  l.append([l_times[i], n_j, n_v, ap_man, ap_vis, g_f, g_t])

# DataFrame resultante
df_ap = pd.DataFrame(l, columns=['Time', 'N°_jogos', 'N°_vitórias', 'Aproveitamento_Mandante', 'Aproveitamento_Visitante', 'Gols_feitos', 'Gols_tomados'])
df_ap

Tendo as características (features) a serem consideradas (df_ap) e os resultados dos jogos (df), pode-se combinar essas informações a fim de classificar cada partida e treinar este classificador.

In [ ]:
# Juntando as informações das duas bases

df_f = pd.merge(df, df_ap, left_on='MANDANTE', right_on='Time')
df_f = pd.merge(df_f, df_ap, left_on='VISITANTE', right_on='Time', suffixes=('_MANDANTE', '_VISITANTE'))

# Definindo o resultado do jogo
df_f['Resultado'] = np.where(df_f['GOLS_MANDANTE'] > df_f['GOLS_VISITANTE'], 'Vitória_MANDANTE', np.where(df_f['GOLS_MANDANTE'] < df_f['GOLS_VISITANTE'], 'Vitória_VISITANTE', 'Empate'))

# Definindo pontuação do jogo
df_f['Pontos Mandante'] = np.where(df_f['GOLS_MANDANTE'] > df_f['GOLS_VISITANTE'], 3, np.where(df_f['GOLS_MANDANTE'] < df_f['GOLS_VISITANTE'], 0, 1))
df_f['Pontos Visitante'] = np.where(df_f['GOLS_MANDANTE'] > df_f['GOLS_VISITANTE'], 0, np.where(df_f['GOLS_MANDANTE'] < df_f['GOLS_VISITANTE'], 3, 1))

In [ ]:
# Ordenar o dataset
df_f = df_f.sort_values(by=['YEAR', 'RODADA', 'MANDANTE', 'VISITANTE'])
df_f = df_f.reset_index(drop=True)
df_f

In [ ]:
# Adicionar a coluna 'ano' nas bases dos scouts
scouts_2014['ano'] = 2014
scouts_2015['ano'] = 2015
scouts_2016['ano'] = 2016
scouts_2017['ano'] = 2017

# Filtrar para considerar apenas os jogadores que participaram da partida (coluna participou é True)
scouts_2014 = scouts_2014[scouts_2014['participou'] == True]
scouts_2016 = scouts_2016[scouts_2016['participou'] == True]

# Filtrar para considerar apenas os jogadores que participaram da partida (Pontuação diferente de 0 - não possue coluna 'participou')
scouts_2015 = scouts_2015[scouts_2015['pontos_num'] != 0]
scouts_2017 = scouts_2017[scouts_2017['pontos_num'] != 0]

# Concatenar as bases de scouts
scouts = pd.concat([scouts_2014, scouts_2015, scouts_2016, scouts_2017], ignore_index=True)

# Retorna o nome do time em relação ao id
scouts = scouts.set_index('clube_id')
clubes = clubes.set_index('id')
scouts = scouts.join(clubes['nome'], on='clube_id')

scouts = scouts[scouts['rodada'] != 0]

In [ ]:
### Pontuação média dos jogadores no ano ###

# Função para calcular a média de pontos dos jogadores no ano
def media_pontos_ano(nome, ano):
    # Filtrar os jogos do ano
    jogos = scouts[(scouts['nome'] == nome) & (scouts['ano'] == int(ano))]

    # Calcular a média de pontos dos jogadores
    media_pontos = jogos['pontos_num'].mean()
    return media_pontos

# Aplicar a função no dataframe df_f para calcular a média de pontos dos times mandante e visitante
df_f['media_pontos_mandante'] = df_f.apply(lambda x: media_pontos_ano(x['MANDANTE'], x['YEAR']), axis=1)
df_f['media_pontos_visitante'] = df_f.apply(lambda x: media_pontos_ano(x['VISITANTE'], x['YEAR']), axis=1)

In [ ]:
### Quantidade de Pontos nos Confrontos Diretos Anteriores ###

# Função para calcular a quantidade de pontos dos confrontos diretos anteriores
def pontos_confronto(time_1, time_2, ano, index_atual):
    # Filtrar os confrontos diretos anteriores
    jogos_m = df_f[(df_f['MANDANTE'] == time_1) & (df_f['VISITANTE'] == time_2) & (df_f['YEAR'] <= ano) & (df_f.index < index_atual)]
    jogos_v = df_f[(df_f['MANDANTE'] == time_2) & (df_f['VISITANTE'] == time_1) & (df_f['YEAR'] <= ano) & (df_f.index < index_atual)]

    # Calcular pontos
    pontos_m = jogos_m['Pontos Mandante'].sum()
    pontos_v = jogos_v['Pontos Visitante'].sum()

    pontos = pontos_m + pontos_v
    return pontos

# Aplicar a função em cada linha do dataframe df_f para calcular a quntidade de pontos dos times mandante e visitante nos confrontos diretos anteriores
df_f['Pontos Confronto Mandante'] = df_f.apply(lambda x: pontos_confronto(x['MANDANTE'], x['VISITANTE'], x['YEAR'], x.name), axis=1)
df_f['Pontos Confronto Visitante'] = df_f.apply(lambda x: pontos_confronto(x['VISITANTE'], x['MANDANTE'], x['YEAR'], x.name), axis=1)

In [ ]:
### Quantidade de Pontos nas Últimas 3 Partidas ###

# Função para calcular a quantidade de pontos dos times nas últimas 3 partidas (caso haja 3 partidas anteriores)
def pontos_ult_3(time, ano, index_atual):
    # Filtrar partidas anteriores
    jogos_m = df_f[(df_f['MANDANTE'] == time) & (df_f['YEAR'] <= ano) & (df_f.index < index_atual)]
    jogos_v = df_f[(df_f['VISITANTE'] == time) & (df_f['YEAR'] <= ano) & (df_f.index < index_atual)]
    jogos_m['Pontos'] = jogos_m['Pontos Mandante']
    jogos_v['Pontos'] = jogos_v['Pontos Visitante']
    jogos = pd.concat([jogos_m, jogos_v])

    # Ordenar por YEAR e RODADA, ambos em ordem decrescente
    jogos = jogos.sort_values(by=['YEAR', 'RODADA'], ascending=[False, False])

    # Selecionar as últimas 3 partidas
    ultimos_jogos = jogos.head(3)

    # Calcular a soma dos pontos
    pontos = ultimos_jogos['Pontos'].sum()

    return pontos

# Aplicar a função em cada linha do dataframe df_f para calcular a quantidade de pontos dos times mandante e visitante nas últimas 3 partidas
df_f['Pontos Ult 3 Mandante'] = df_f.apply(lambda x: pontos_ult_3(x['MANDANTE'], x['YEAR'], x.name), axis=1)
df_f['Pontos Ult 3 Visitante'] = df_f.apply(lambda x: pontos_ult_3(x['VISITANTE'], x['YEAR'], x.name), axis=1)

Antes de iniciar a classificação e treinamentos, pode-se observar o dataset final:

In [ ]:
df_f

## Classificador

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder

# Definição das features
feat = ['Aproveitamento_Mandante_MANDANTE', 'Aproveitamento_Visitante_MANDANTE', 'Gols_feitos_MANDANTE', 'Gols_tomados_MANDANTE', 'Aproveitamento_Mandante_VISITANTE', 'Aproveitamento_Visitante_VISITANTE', 'Gols_feitos_VISITANTE', 'Gols_tomados_VISITANTE', 'media_pontos_mandante', 'media_pontos_visitante', 'Pontos Confronto Mandante', 'Pontos Confronto Visitante', 'Pontos Ult 3 Mandante', 'Pontos Ult 3 Visitante']
#feat = ['Aproveitamento_Mandante_MANDANTE', 'Gols_feitos_VISITANTE']
# Divisão dos dados em treino e teste através do train_test_split
X = df_f[feat]
y = le.fit_transform(df_f['Resultado'])

# Convertendo a coluna de resultados para valores numéricos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Logistic Regression

In [ ]:
# Treinando o modelo LogisticRegression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train, y_train)

# Desempenho do modelo LogisticRegression
y_pred = log_reg.predict(X_test)
ac = accuracy_score(y_test, y_pred)
r_class = classification_report(y_test, y_pred)

print(f'Acurácia: {ac:.2f}')
print('Relatório de Classificação:\n', r_class)

### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Treinando o modelo DecisionTree
d_tree = DecisionTreeClassifier(random_state=42)
d_tree.fit(X_train, y_train)

# Desempenho do modelo DecisionTree
y_pred_d_tree = d_tree.predict(X_test)
ac_d_tree = accuracy_score(y_test, y_pred_d_tree)
r_d_tree = classification_report(y_test, y_pred_d_tree)

print(f'Acurácia: {ac_d_tree:.2f}')
print("Relatório de Classificação:\n", r_d_tree)

### Matriz de Confusão (Logistic Regression e Decision Tree)

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt

# Matriz de confusão
def plot_confusion_matrix(ax, y_true, y_pred, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", ax=ax, vmin=0, vmax=100)  # Ajuste as escalas de cor aqui
    ax.set_title(title)
    ax.set_xlabel('Previsão do Modelo')
    ax.set_ylabel('Verdadeiro Resultado')

fig, axs = plt.subplots(1, 2, figsize=(12, 4))

# Matriz de confusão para LogisticRegression
plot_confusion_matrix(axs[0], y_test, y_pred, 'Matriz de Confusão - Regressão Logística')

# Matriz de confusão para DecisionTree
plot_confusion_matrix(axs[1], y_test, y_pred_d_tree, 'Matriz de Confusão - Árvore de Decisão')

plt.tight_layout()
plt.show()

### SVM

In [ ]:
from sklearn.svm import SVC

svm_model = SVC(kernel='linear', random_state=42)

# Treino
svm_model.fit(X_train, y_train)
y_pred_svm = svm_model.predict(X_test)

# Desempenho
ac_svm = accuracy_score(y_test, y_pred_svm)
r_svm = classification_report(y_test, y_pred_svm)

print(f'Acurácia: {ac_svm:.2f}')
print('Relatório de Classificação:\n', r_svm)

### RandomForestClassifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Treino
rf_model.fit(X_train, y_train)
y_pred_rf = rf_model.predict(X_test)

# Desempenho
ac_rf = accuracy_score(y_test, y_pred_rf)
r_rf = classification_report(y_test, y_pred_rf)

print(f'Acurácia: {ac_rf:.2f}')
print('Relatório de Classificação:\n', r_rf)

### MLPClassifier

In [ ]:
from sklearn.neural_network import MLPClassifier
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, alpha=0.0001,
                     solver='adam', verbose=10,  random_state=42,tol=0.000000001)

# Treino
mlp_model.fit(X_train, y_train)
y_pred_mlp = mlp_model.predict(X_test)

# Desempenho
ac_mlp = accuracy_score(y_test, y_pred_mlp)
r_mlp = classification_report(y_test, y_pred_mlp)

print(f'Acurácia: {ac_mlp:.2f}')
print('Relatório de Classificação:\n', r_mlp)

### Validação Cruzada

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

# Definição das features
feat = ['Aproveitamento_Mandante_MANDANTE', 'Aproveitamento_Visitante_MANDANTE', 'Gols_feitos_MANDANTE', 'Gols_tomados_MANDANTE', 'Aproveitamento_Mandante_VISITANTE', 'Aproveitamento_Visitante_VISITANTE', 'Gols_feitos_VISITANTE', 'Gols_tomados_VISITANTE', 'media_pontos_mandante', 'media_pontos_visitante', 'Pontos Confronto Mandante', 'Pontos Confronto Visitante', 'Pontos Ult 3 Mandante', 'Pontos Ult 3 Visitante']

X = df_f[feat]
y = df_f['Resultado']

# Número de folds
num_folds = 5

# Modelo de LogisticRegression
log_reg = LogisticRegression(random_state=42)
scores_log_reg = cross_val_score(log_reg, X, y, cv=num_folds, scoring='accuracy')

# Modelo de Árvore de Decisão
d_tree = DecisionTreeClassifier(random_state=42)
scores_d_tree = cross_val_score(d_tree, X, y, cv=num_folds, scoring='accuracy')

# Resultados
print(f'(Validação Cruzada) Regressão Logística - Acurácia média: {scores_log_reg.mean():.2f}')
print(f'(Validação Cruzada) Árvore de Decisão - Acurácia média: {scores_d_tree.mean():.2f}')

## Teste das combinações de features

Serão em torno de 2047 combinações possíveis de features para cada modelo.

**65532 combinações**

In [ ]:
pip install -U scikit-learn

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
import itertools

le = LabelEncoder()
scaler = StandardScaler()

# Lista de features
feat = [
    'Aproveitamento_Mandante_MANDANTE', 'Aproveitamento_Visitante_MANDANTE', 
    'Gols_feitos_MANDANTE', 'Gols_tomados_MANDANTE',
    'Aproveitamento_Mandante_VISITANTE', 'Aproveitamento_Visitante_VISITANTE', 
    'Gols_feitos_VISITANTE', 'Gols_tomados_VISITANTE',
    'media_pontos_mandante', 'media_pontos_visitante', 
    'Pontos Confronto Mandante', 'Pontos Confronto Visitante', 
    'Pontos Ult 3 Mandante', 'Pontos Ult 3 Visitante'
]

# Função para analisar combinações de features para um único modelo
def analisar_modelo(modelo, nome_modelo, feat, df_f, le):
    print(f"Analisando modelo: {nome_modelo}")
    results = []
    i = 0
    
    for r in range(1, len(feat) + 1):
        for feat_comb in itertools.combinations(feat, r):
            X = df_f[list(feat_comb)]
            y = le.fit_transform(df_f['Resultado'])

            # Divisão dos dados em treino e teste
            X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

            # Padronizando as features para LogisticRegression, SVM e MLP
            if nome_modelo in ['Logistic Regression', 'SVM', 'MLP']:
                X_train = scaler.fit_transform(X_train)
                X_test = scaler.transform(X_test)
                
            # Treinando o modelo
            modelo.fit(X_train, y_train)

            # Prevendo no conjunto de teste
            y_pred = modelo.predict(X_test)

            # Calculando a acurácia
            accuracy = accuracy_score(y_test, y_pred)
            f1 = f1_score(y_test, y_pred, average='weighted')
            i += 1

            # Armazenando os resultados
            results.append({'Modelo': nome_modelo, 'Features': feat_comb, 'Num_Features': len(feat_comb), 'Accuracy': accuracy, 'F1-Score': f1})
            print(f"Teste {i}: {nome_modelo} com {len(feat_comb)} features. Acurácia: {accuracy:.4f}, F1-Score: {f1:.4f}")

    results_df = pd.DataFrame(results)
    return results_df


In [ ]:
# Modelos individuais
modelos = [('Logistic Regression', LogisticRegression(
        multi_class='multinomial', 
        solver='lbfgs', 
        max_iter=500, 
        C=1.0, 
        class_weight='balanced'
    )),
    ('Decision Tree', DecisionTreeClassifier(
        max_depth=10, 
        min_samples_split=20, 
        min_samples_leaf=10, 
        random_state=42, 
        class_weight='balanced'
    )),
    ('Random Forest', RandomForestClassifier(
        n_estimators=100, 
        max_depth=10, 
        min_samples_split=20, 
        min_samples_leaf=10, 
        random_state=42, 
        class_weight='balanced'
    )),
    ('SVM', SVC(
        kernel='rbf', 
        C=1.0, 
        gamma='scale', 
        random_state=42, 
        class_weight='balanced'
    )),
    ('MLP', MLPClassifier(
        hidden_layer_sizes=(50, 30), 
        max_iter=500, 
        solver='adam', 
        alpha=0.0001, 
        random_state=42, 
        tol=1e-4
    ))
]

In [ ]:
# Logistic Regression

df_logreg = analisar_modelo(modelos[0][1], modelos[0][0], feat, df_f, le)

In [ ]:
import pandas as pd
df_logreg.to_csv('logreg.csv', index=False)

In [ ]:
# Decision Tree

df_dectree = analisar_modelo(modelos[1][1], modelos[1][0], feat, df_f, le)

In [ ]:
df_dectree.to_csv('dectree.csv', index=False)

In [ ]:
# Random Forest

df_randfor = analisar_modelo(modelos[2][1], modelos[2][0], feat, df_f, le)

In [ ]:
df_randfor.to_csv('randfor.csv', index=False)

In [ ]:
# SVM

df_svm = analisar_modelo(modelos[3][1], modelos[3][0], feat, df_f, le)

In [ ]:
df_svm.to_csv('svm.csv', index=False)

In [ ]:
# MLP

df_mlp = analisar_modelo(modelos[4][1], modelos[4][0], feat, df_f, le)

In [ ]:
df_mlp.to_csv('mlp.csv', index=False)